<a href="https://colab.research.google.com/github/nxxk23/kidney/blob/main/impute_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
table = pd.read_csv('/content/drive/MyDrive/kidney/cut_outlier.csv')
table = table.drop(['Unnamed: 0'], axis=1)
table.head()

,PatientUID,PatientVisitUID,CWhen,eGFR,datetime,PreviousDateTime,Sex,Age,ENTY,NATNL,...,G4M2_Febuxostat,G3M2_Ezetimibe,G1M4_Non-dihydropyridine_CCB,G5M1_Ketoanalog_amino_acid,G2M7_GLP1_RA,G4M3_Uricosuric_agent,LabLocation,stage,stage_latest,Fold
0,2299,3386255,2559-08-15 09:51:54.913,71.7,2016-08-15 09:51:54.913,NaN,หญิง,73.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
1,2299,3640924,2559-11-11 09:40:06.053,66.8,2016-11-11 09:40:06.053,2016-08-15 09:51:54.913,หญิง,73.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
2,2299,4020970,2560-03-31 10:18:54.363,67.2,2017-03-31 10:18:54.363,2016-11-11 09:40:06.053,หญิง,74.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
3,2299,4286612,2560-07-05 10:40:40.763,69.2,2017-07-05 10:40:40.763,2017-03-31 10:18:54.363,หญิง,74.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
4,2299,4546471,2560-10-03 09:19:54.710,63.6,2017-10-03 09:19:54.710,2017-07-05 10:40:40.763,หญิง,74.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0


In [12]:
data = table.copy()

In [14]:
# Find columns with null values in data
columns_with_null = data.columns[data.isnull().any()].tolist()

# Find columns to keep in data
columns_to_keep = [col for col in data.columns if col not in columns_with_null]

# Display the original order of columns
print("Original order of columns in data:")
print(data.columns.tolist())

# Display columns with null values and columns to keep
print("\nColumns with null values:")
print(columns_with_null)
print("\nColumns to keep:")
print(columns_to_keep)

Original order of columns in data:
['PatientUID', 'PatientVisitUID', 'CWhen', 'eGFR', 'datetime', 'PreviousDateTime', 'Sex', 'Age', 'ENTY', 'NATNL', 'VisitLocation', 'StartDTTM', 'EndDttm', 'InPatient', 'OutPatient', 'Emergency', 'BMI', 'BSA', 'Diastolic BP', 'Systolic BP', 'L1_eGFR', 'L10_Phosphorus', 'L11_Albumin', 'L12_Hemoglobin', 'L13_Hematocrit', 'L14_WBC', 'L15_Platelet', 'L16_TIBC', 'L18_Ferritin', 'L19_Blood_sugar', 'L2_BUN', 'L20_HbA1c', 'L21_Cholesterol', 'L22_Triglyceride', 'L23_LDL_Cholesterol', 'L24_HDL_Cholesterol', 'L25_Parathyroid_hormone', 'L26_Vitamin_D', 'L27_UrineAlbumin/Cr_ratio', 'L28_UrineProtein/Cr_ratio', 'L29_UrineVolume_24hrs', 'L3_Creatinine', 'L30_UrineUrea_24hrs', 'L32_UrineProtein_24hrs', 'L4_Sodium', 'L5_Potassium', 'L6_Chloride', 'L7_Bicarbonate', 'L8_Uric', 'L9_Calcium', 'D1_Hypertension', 'D10_Ischemic_heart_disease', 'D11_Heart_failure', 'D12_Thalassemia', 'D13_Kidney_stone', 'D14_Valvular_heart_disease', 'D15_Solid_malignancy', 'D16_Hematologic_mal

## **Add One Binary Variable**

In [16]:
import numpy as np

for col in columns_with_null:
    data['new' + col] = np.where(data[col].isnull(), 1, 0)

In [17]:
# Select columns that start with 'new'
new_columns = [col for col in data.columns if col.startswith('new')]

# Select columns with null values and 'new' columns
selected_columns = columns_with_null + new_columns

## **impute**

    * Mode = If the data type is 'object'.
    * Mean = If the data type is numeric (not 'object') and dont have outlier
    * Median = If the data type is numeric (not 'object') and have outlier

In [24]:
missing_for_binary = pd.concat([data[columns_to_keep], data[columns_with_null]], axis=1)
imputed_table = missing_for_binary.copy()
# Print information before imputation
print("Null values before imputation:")
print(imputed_table[columns_with_null].info())

Null values before imputation:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318675 entries, 0 to 318674
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   PreviousDateTime           318674 non-null  object 
 1   BMI                        226601 non-null  float64
 2   BSA                        227020 non-null  float64
 3   Diastolic BP               279283 non-null  float64
 4   Systolic BP                278487 non-null  float64
 5   L10_Phosphorus             62811 non-null   float64
 6   L11_Albumin                149765 non-null  float64
 7   L12_Hemoglobin             153033 non-null  float64
 8   L13_Hematocrit             200048 non-null  float64
 9   L14_WBC                    199375 non-null  float64
 10  L15_Platelet               199038 non-null  float64
 11  L16_TIBC                   5890 non-null    float64
 12  L18_Ferritin               11881 non-null   float64
 13

In [25]:
# List of columns without outliers
columns_without_outliers = ['L28_UrineProtein/Cr_ratio', 'L21_Cholesterol', 'L19_Blood_sugar', 'L12_Hemoglobin', 'L13_Hematocrit', 'L5_Potassium', 'L3_Creatinine']

# Identify all columns in the dataset
all_columns = data.columns.tolist()

# Identify columns with outliers by subtracting columns_without_outliers from all_columns
columns_with_outliers = list(set(all_columns) - set(columns_without_outliers))

# Exclude datetime columns
datetime_columns = data.select_dtypes(include=['object']).columns.tolist()

# Exclude datetime columns from columns_with_null
columns_with_null = [col for col in columns_with_null if col not in datetime_columns]

print("Columns with outliers:", columns_with_outliers)

Columns with outliers: ['BSA', 'G1M1_ACEI', 'newL21_Cholesterol', 'G3M2_Ezetimibe', 'newPreviousDateTime', 'newL19_Blood_sugar', 'CWhen', 'G5M2_NSAIDs', 'Age', 'VisitLocation', 'L18_Ferritin', 'G3M3_Fibrate', 'newL4_Sodium', 'newL9_Calcium', 'datetime', 'L26_Vitamin_D', 'newL29_UrineVolume_24hrs', 'L22_Triglyceride', 'G1M6_MRA', 'newL20_HbA1c', 'L15_Platelet', 'L4_Sodium', 'PreviousDateTime', 'D3_Dyslipidemia', 'newL10_Phosphorus', 'newL8_Uric', 'L16_TIBC', 'L11_Albumin', 'G1M9_Vasodilator', 'newL16_TIBC', 'L30_UrineUrea_24hrs', 'newBSA', 'L32_UrineProtein_24hrs', 'newL23_LDL_Cholesterol', 'G2M3_DPP4_inhibitor', 'newL7_Bicarbonate', 'newL14_WBC', 'D12_Thalassemia', 'newL28_UrineProtein/Cr_ratio', 'L6_Chloride', 'newL24_HDL_Cholesterol', 'D13_Kidney_stone', 'newL32_UrineProtein_24hrs', 'G4M1_Allopurinol', 'G1M7_Beta_blocker', 'G3M1_Statin', 'newL18_Ferritin', 'BMI', 'newL3_Creatinine', 'ENTY', 'L25_Parathyroid_hormone', 'D4_SLE', 'D1_Hypertension', 'eGFR', 'D16_Hematologic_malignancy', 

In [26]:
# Check for outliers in each column and fill missing values accordingly
for column in columns_with_null:
    if column in columns_with_outliers:
        # Columns with outliers
        median_values_patient = imputed_table.groupby('PatientUID')[column].median()
        imputed_table[column].fillna(imputed_table['PatientUID'].map(median_values_patient), inplace=True)
    else:
        # Columns without outliers, use mean for each patient
        mean_values_patient = imputed_table.groupby('PatientUID')[column].mean()
        imputed_table[column].fillna(imputed_table['PatientUID'].map(mean_values_patient), inplace=True)

# Fill remaining missing values using overall mean (or median)
for column in columns_with_null:
    if imputed_table[column].isnull().any():
        if imputed_table[column].dtype == 'O':  # Check dtype for the specific column
            # Use overall mode for remaining missing values
            imputed_table[column].fillna(imputed_table[column].mode()[0], inplace=True)
        else:
            # Use overall mean for remaining missing values
            overall_mean = imputed_table[column].mean()
            imputed_table[column].fillna(overall_mean, inplace=True)

# Print information after imputation
print("\nNull values after imputation:")
print(imputed_table[columns_with_null].isnull().sum())


Null values after imputation:
BMI                          0
BSA                          0
Diastolic BP                 0
Systolic BP                  0
L10_Phosphorus               0
L11_Albumin                  0
L12_Hemoglobin               0
L13_Hematocrit               0
L14_WBC                      0
L15_Platelet                 0
L16_TIBC                     0
L18_Ferritin                 0
L19_Blood_sugar              0
L2_BUN                       0
L20_HbA1c                    0
L21_Cholesterol              0
L22_Triglyceride             0
L23_LDL_Cholesterol          0
L24_HDL_Cholesterol          0
L25_Parathyroid_hormone      0
L26_Vitamin_D                0
L27_UrineAlbumin/Cr_ratio    0
L28_UrineProtein/Cr_ratio    0
L29_UrineVolume_24hrs        0
L3_Creatinine                0
L30_UrineUrea_24hrs          0
L32_UrineProtein_24hrs       0
L4_Sodium                    0
L5_Potassium                 0
L6_Chloride                  0
L7_Bicarbonate               0
L8_Uric 

In [31]:
# Print the number of null values in each column
null_counts = imputed_table.isnull().sum()
# Get columns with null values
columns_with_null = null_counts[null_counts > 0].index.tolist()
print("\nColumns with null values:")
print(columns_with_null)


Columns with null values:
['PreviousDateTime']


In [33]:
# Select columns that start with 'new'
columns_with_new = [col for col in data.columns if col.startswith('new')]

# Merge the selected columns
merged_data = pd.concat([imputed_table, data[columns_with_new]], axis=1)
merged_data.head()

,PatientUID,PatientVisitUID,CWhen,eGFR,datetime,Sex,Age,ENTY,NATNL,VisitLocation,...,newL29_UrineVolume_24hrs,newL3_Creatinine,newL30_UrineUrea_24hrs,newL32_UrineProtein_24hrs,newL4_Sodium,newL5_Potassium,newL6_Chloride,newL7_Bicarbonate,newL8_Uric,newL9_Calcium
0,2299,3386255,2559-08-15 09:51:54.913,71.7,2016-08-15 09:51:54.913,หญิง,73.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,1,0,1,1,1
1,2299,3640924,2559-11-11 09:40:06.053,66.8,2016-11-11 09:40:06.053,หญิง,73.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,0,1,1,1,1
2,2299,4020970,2560-03-31 10:18:54.363,67.2,2017-03-31 10:18:54.363,หญิง,74.0,Out Patient,ไทย,ออร์โธปิดิกส์ (OPD)-D09,...,1,0,1,1,0,0,0,1,1,1
3,2299,4286612,2560-07-05 10:40:40.763,69.2,2017-07-05 10:40:40.763,หญิง,74.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,0,1,1,0,1,1,1
4,2299,4546471,2560-10-03 09:19:54.710,63.6,2017-10-03 09:19:54.710,หญิง,74.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,1,0,1,1,1


In [34]:
binary_columns = [col for col in merged_data.columns if col.startswith('new')]
#data.drop(binary_columns, axis=1, inplace=True)
merged_data

,PatientUID,PatientVisitUID,CWhen,eGFR,datetime,Sex,Age,ENTY,NATNL,VisitLocation,...,newL29_UrineVolume_24hrs,newL3_Creatinine,newL30_UrineUrea_24hrs,newL32_UrineProtein_24hrs,newL4_Sodium,newL5_Potassium,newL6_Chloride,newL7_Bicarbonate,newL8_Uric,newL9_Calcium
0,2299,3386255,2559-08-15 09:51:54.913,71.70,2016-08-15 09:51:54.913,หญิง,73.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,1,0,1,1,1
1,2299,3640924,2559-11-11 09:40:06.053,66.80,2016-11-11 09:40:06.053,หญิง,73.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,0,1,1,1,1
2,2299,4020970,2560-03-31 10:18:54.363,67.20,2017-03-31 10:18:54.363,หญิง,74.0,Out Patient,ไทย,ออร์โธปิดิกส์ (OPD)-D09,...,1,0,1,1,0,0,0,1,1,1
3,2299,4286612,2560-07-05 10:40:40.763,69.20,2017-07-05 10:40:40.763,หญิง,74.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,0,1,1,0,1,1,1
4,2299,4546471,2560-10-03 09:19:54.710,63.60,2017-10-03 09:19:54.710,หญิง,74.0,Out Patient,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318670,3432056,10312896,2566-01-30 16:16:45.013,72.86,2023-01-30 16:16:45.013,หญิง,74.0,InPatient,ไทย,หอผู้ป่วย 5ง-5DW,...,1,0,1,1,0,0,0,1,0,1
318671,3432056,10408954,2566-02-24 21:05:12.747,68.69,2023-02-24 21:05:12.747,หญิง,74.0,InPatient,ไทย,หอผู้ป่วย 5ง-5DW,...,1,0,1,1,0,0,0,1,0,1
318672,3432056,10573233,2566-04-10 09:51:32.587,45.08,2023-04-10 09:51:32.587,หญิง,74.0,InPatient,ไทย,หอผู้ป่วย 5ก-5AW,...,1,0,1,1,0,0,0,1,1,1
318673,3437526,10191122,2565-12-26 09:29:11.307,55.94,2022-12-26 09:29:11.307,ชาย,67.0,InPatient,ไทย,หอผู้ป่วย 3ก-3AW,...,1,0,1,1,0,0,0,1,1,1


In [35]:
data = merged_data.copy()

## **encode object column**

In [36]:
data['LabLocation'] = np.where(data['LabLocation'] == "ห้องปฏิบัติการจุลทรรศน์วินิจฉัย", 1, 0)
data['LabLocation'].value_counts()

LabLocation
1    174284
0    144391
Name: count, dtype: int64

In [40]:
nationality_map = {'ไทย': 0}

caucasian = ['บริติช  (อังกฤษ, สก็อตแลนด์)', 'อเมริกัน', 'เยอรมัน', 'โอมาน', 'ออสเตรเลีย', 'แคนาดา',
             'อิตาลี', 'สวิส', 'ซีเรีย', 'เดนมาร์ก', 'นอร์เวย์', 'ดัตช์', 'สเปน', 'เบลเยียม', 'ไอซ์แลนด์',
             'ฝรั่งเศส', 'เซนต์คิตส์และเนวิส', 'สวีเดน', 'ออสเตรีย', 'ไซปรัส']
asian = ['ลาว','จีน','เวียดนาม','เขมรอพยพ','ไทยลื้อ','ญี่ปุ่น', 'จีน(ไต้หวัน)', 'พม่า']
other = ['ไม่ระบุ','อื่นๆ','ไร้สัญชาติ']

for nationality in asian:
    nationality_map[nationality] = 0

for nationality in other:
    nationality_map[nationality] = 0

for nationality in caucasian:
    nationality_map[nationality] = 1

data['NATNL'] = data['NATNL'].replace(nationality_map)
data['NATNL'].value_counts()
#asian: 0, non-asian:1

NATNL
0    317607
1      1068
Name: count, dtype: int64

In [41]:
data['Sex'] = data['Sex'].replace({'ชาย': 0, 'หญิง': 1})
data['Sex'].value_counts()

Sex
0    171458
1    147217
Name: count, dtype: int64

In [43]:
data

,PatientUID,PatientVisitUID,CWhen,eGFR,datetime,Sex,Age,ENTY,NATNL,VisitLocation,...,newL29_UrineVolume_24hrs,newL3_Creatinine,newL30_UrineUrea_24hrs,newL32_UrineProtein_24hrs,newL4_Sodium,newL5_Potassium,newL6_Chloride,newL7_Bicarbonate,newL8_Uric,newL9_Calcium
0,2299,3386255,2559-08-15 09:51:54.913,71.70,2016-08-15 09:51:54.913,1,73.0,Out Patient,0,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,1,0,1,1,1
1,2299,3640924,2559-11-11 09:40:06.053,66.80,2016-11-11 09:40:06.053,1,73.0,Out Patient,0,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,0,1,1,1,1
2,2299,4020970,2560-03-31 10:18:54.363,67.20,2017-03-31 10:18:54.363,1,74.0,Out Patient,0,ออร์โธปิดิกส์ (OPD)-D09,...,1,0,1,1,0,0,0,1,1,1
3,2299,4286612,2560-07-05 10:40:40.763,69.20,2017-07-05 10:40:40.763,1,74.0,Out Patient,0,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,0,1,1,0,1,1,1
4,2299,4546471,2560-10-03 09:19:54.710,63.60,2017-10-03 09:19:54.710,1,74.0,Out Patient,0,เวชปฏิบัติทั่วไป (OPD)-D03,...,1,0,1,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318670,3432056,10312896,2566-01-30 16:16:45.013,72.86,2023-01-30 16:16:45.013,1,74.0,InPatient,0,หอผู้ป่วย 5ง-5DW,...,1,0,1,1,0,0,0,1,0,1
318671,3432056,10408954,2566-02-24 21:05:12.747,68.69,2023-02-24 21:05:12.747,1,74.0,InPatient,0,หอผู้ป่วย 5ง-5DW,...,1,0,1,1,0,0,0,1,0,1
318672,3432056,10573233,2566-04-10 09:51:32.587,45.08,2023-04-10 09:51:32.587,1,74.0,InPatient,0,หอผู้ป่วย 5ก-5AW,...,1,0,1,1,0,0,0,1,1,1
318673,3437526,10191122,2565-12-26 09:29:11.307,55.94,2022-12-26 09:29:11.307,0,67.0,InPatient,0,หอผู้ป่วย 3ก-3AW,...,1,0,1,1,0,0,0,1,1,1


In [42]:
data.to_csv('/content/drive/MyDrive/kidney/binary.csv')